# Fonction de Rosenbrock généralisée

In [ ]:
using LinearAlgebra
using ForwardDiff
using BenchmarkTools

Plusieurs généralisations à $n$ dimensions de la fonction de Rosenbrock ont été proposées dans la littérature. Nous considérons ici la généralisation définie par:
$$
f(x) = \sum_{i = 1}^{n-1} \left( 100(x_{(i+1)}^2-x_{(i)})^2 + (x_{(i)}-1)^2 \right).
$$
Il s'agit de la somme de $n-1$ fonctions de Rosenbrock à 2 dimensions.

In [ ]:
rosenbrock(x::Vector) = sum(100 * (x[i+1]^2 - x[i])^2 + (x[i] - 1)^2 for i in 1:length(x)-1)

Le gradient est
$$
\nabla_x f(x) =
\begin{pmatrix}
-200(x_{(2)}^2-x_{(1)})+2(x_{(1)}-1) \\
400x_{(2)}(x_{(2)}^2-x_{(1)})-200(x_{(3)}^2-x_{(2)})+2(x_{(2)}-1) \\
\vdots \\
400x_{(i)}(x_{(i)}^2-x_{(i-1)})-200(x_{(i+1)}^2-x_{(i)})+2(x_{(i)}-1) \\
\vdots \\
400x_{(n-1)}(x_{(n-1)}^2-x_{(n-2)})-200(x_{(n-1)}^2-x_{(n-2)})+2(x_{(n-2)}-1) \\
400x_{(n)}(x_{(n)}^2-x_{(n-1)})
\end{pmatrix}.
$$

In [ ]:
function ∇f(x:: Vector)
    n = length(x)
    g = zeros(n)
    for i = 1:n-1
        g[i] = -200*(x[i+1]^2-x[i])+2*(x[i]-1)
    end
    for i = 2:n
        g[i] += 400*x[i]*(x[i]^2-x[i-1])
    end
    return g
end

La matrice hessienne est
$$
\nabla_{xx}^2 f(x) =
\begin{pmatrix}
202 & -400x_{(2)} & 0 & 0 & \cdots & 0 & 0 & 0 \\
 -400x_{(2)} & 400(x_{(2)}^2-x(1))+800x_{(2)}^2+202 & -400x_{(3)} & 0 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots &\vdots &\vdots & \ddots & \vdots &\vdots &\vdots &\\
0 & 0 & 0 & 0 & \cdots & -400x_{(n-2)} & 400(x_{(n-1)}^2-x_{(n-2)})+800x_{(n-1)}^2 +202 & -400x_{(n-1)} \\
0 & 0 & 0 & 0 & \cdots & 0 & -400x_{(n-1)} & 400(x_{(n)}^2-x_{(n-1)})+800x_{(n)}^2
\end{pmatrix}.
$$

In [ ]:
function Hess(x:: Vector)
    n = length(x)
    H = zeros(n,n)
    H[1,1] = 202
    for i = 2:n
        H[i,i-1] = H[i-1,i] = -400*x[i]
        H[i,i] = 400*(x[i]^2-x[i-1])+800*x[i]^2 + 202
    end
    H[n,n] -= 202
    return H
end

Les calculs sont complexes! Nous allons les vérifier à l'aide de la différentiation automatique.

In [ ]:
g = x -> ForwardDiff.gradient(rosenbrock, x);
H = x -> ForwardDiff.hessian(rosenbrock, x);
function g!(storage::Vector, x::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end
function H!(storage::Matrix, x::Vector)
    s = H(x)
    n, m = size(s)
    storage[1:n,1:m] = s[1:length(s)]
end

In [ ]:
n = 1000
x = ones(n)/4;

In [ ]:
rosenbrock(x)

Comparons les gradients en matrices hessiennes en $x$.

In [ ]:
norm(g(x)-∇f(x))

In [ ]:
norm(H(x)-Hess(x))

Nous pouvons de plus remarquer que la matrice hessienne est creuse.

In [ ]:
H(x)

La différentiation automatique est-elle efficace?

In [ ]:
@benchmark H(x)

In [ ]:
@benchmark Hess(x)

On voit qu'il y a avantage a utiliser l'implémentation exacte de la matrice hessienne, mais est-ce dû au caractère creux de la matrice?

In [ ]:
using SparseArrays

In [ ]:
He = Hess(x)
SH = sparse(He)

Comme ce qui nous intéresse est le produit matrice-vecteur, regardons le gain potentiel obtenu avec le stockage creux.

In [ ]:
@benchmark SH*x

In [ ]:
@benchmark He*x

Plutôt que de construire la matrice dense puis de la réduire à une matrice creuse, nous pouvons directement utiliser les propriétés de la matrice pour exploiter sa structure, par exemple en exploitant le fait qu'il s'agit d'une matrice tridiagonale symétrique.

In [ ]:
function TriHess(x)
    n = length(x)
    d = zeros(n)
    d[1] = 202
    d[2:n] = [400*(x[i]^2-x[i-1])+800*x[i]^2 + 202 for i = 2:n]
    d[n] -= 202
    dl = [-400*x[i] for i = 2:n]
    H = SymTridiagonal(d, dl) # H est déclarée comme matrice tridiagonale symétrique.
    return H
end

In [ ]:
T = TriHess(x)
@benchmark T*x

Cela permet de monter la taille du système, qui sinon serait ingérable. Prenons par exemple une dimension de un million.

In [ ]:
x = ones(1000000)/4
T = TriHess(x)
@benchmark T*x

Revenons à un exemple plus modeste.

In [ ]:
x = ones(1000)/4
v = copy(x);
rosenbrock(x);

Considérons le produit entre le gradient évalué en $x$ et un vecteur $v$.

In [ ]:
gvr = x -> dot(g(x), v)
@benchmark gvr(x)

Nous pouvons à nouveau être plus efficace.

In [ ]:
function gvr2(x:: Vector)
    n = length(x)
    t = (-200*(x[2]^2-x[1])+2*(x[1]-1))*v[1]
    for i = 2:n-1
        t += (-200*(x[i+1]^2-x[i])+2*(x[i]-1)+400*x[i]*(x[i]^2-x[i-1]))*v[i]
    end
    t += 400*x[n]*(x[n]^2-x[n-1])*v[n]
    return t
end
@benchmark gvr2(x)

Les deux approches donnent pourtant le même résultat.

In [ ]:
norm(gvr(x)-gvr2(x))

Le produit matrice vecteur entre $H$ et $v$ peut être calculé en calculant le gradient du produit scalaire entre le gradient et $v$. En prenant les deux implémentations de ce produit scalaire, cela donne

In [ ]:
Hv = x -> ForwardDiff.gradient(gvr, x)
norm(Hv(x)-TriHess(x)*v)

In [ ]:
Hv2 = x -> ForwardDiff.gradient(gvr2, x)
norm(Hv2(x)-TriHess(x)*v)

Comparons les temps de calcul des différentes approches.

In [ ]:
@benchmark TriHess(x)*v

In [ ]:
@benchmark Hess(x)*v

In [ ]:
@benchmark Hv(x)

In [ ]:
@benchmark Hv2(x)

On voit qu'exploiter la structure de la matrice est plus efficace! Une grande part de la sous-performance vient de la nom prise en compte du caractère creux par la différentiation automatique, et on gagne à implémenter des calculs exacts.

## Inversion de matrices creuses

Si la matrice hessienne est creuse, son inverse peut être dense. Il n'est dès lors pas pratique d'utiliser des méthodes basées sur l'inverse de la matrice hessienne.

In [ ]:
H = TriHess(x)

In [ ]:
inv(H)

Factoriser la matrice n'implique pas le même niveau de remplissage. Dans le cas présent, il est remarquable de noter que la structure creuse est parfaitement respectée.

In [ ]:
factorize(H)

Cela permet à nouveau de privilégier la résolutions de systèmes linéaires par factorisation à l'inversion de matrice, mais l'effort de calcul pour la factorisation reste significatif.

In [ ]:
@benchmark factorize(H)

In [ ]:
@benchmark H\v

Malheureusement, tous les problèmes ne présentent pas une structure aussi intéressante. Néanmoins, l'implémentation directe du produit matrice-vecteur permet de contourner un autre problème: le stockage des matrices en grande dimension. Reprenons le stockage dense. On a

In [ ]:
@benchmark Hess(x)\v

In [ ]:
x = ones(100000)/4
v = copy(x);

In [ ]:
@benchmark Hess(x)\v

In [ ]:
@benchmark TriHess(x)\v

In [ ]:
@benchmark Hv2(x)

Lorsque le nombre de variables devient trop grand, la résolution directe du système linéaire avec une matrice dense est prohibitive (l'exécution du code a pu résulter sur une erreur de débordement mémoire), alors qu'il est raisonnable de calculer le produit matrice-vecteur pour quelques itérations, ouvrant la voie aux méthodes de gradient conjugué tronqués. Exploiter la structure reste cependant à nouveau plus efficace.